In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from uszipcode import SearchEngine
from timezonefinder import TimezoneFinder

In [2]:
a_data = pd.read_csv(r'C:\Users\Mikhail\Documents\Data 606\US_Accidents_Dec19.csv')
a_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 49 columns):
ID                       1048575 non-null object
Source                   1048575 non-null object
TMC                      1048575 non-null int64
Severity                 1048575 non-null int64
Start_Time               1048575 non-null object
End_Time                 1048575 non-null object
Start_Lat                1048575 non-null float64
Start_Lng                1048575 non-null float64
End_Lat                  0 non-null float64
End_Lng                  0 non-null float64
Distance(mi)             1048575 non-null float64
Description              1048575 non-null object
Number                   406187 non-null float64
Street                   1048575 non-null object
Side                     1048575 non-null object
City                     1048542 non-null object
County                   1048575 non-null object
State                    1048575 non-null object
Zipcode       

In [3]:
#Data cleaning
a_data.drop(['End_Lat', 'End_Lng', 'ID', 'Number'], axis = 1, inplace = True)
a_data['Start_Time'] = pd.to_datetime(a_data['Start_Time'])
a_data['End_Time'] = pd.to_datetime(a_data['End_Time'])
a_data['Time_Elapsed'] = a_data['End_Time'] - a_data['Start_Time']
a_data['Weather_Timestamp'] = pd.to_datetime(a_data['Weather_Timestamp'])
a_data['Weather_Condition'].fillna('Indeterminate', inplace = True)
a_data.loc[a_data['State'] == 'DC', 'City'] = 'Washington'

a_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 46 columns):
Source                   1048575 non-null object
TMC                      1048575 non-null int64
Severity                 1048575 non-null int64
Start_Time               1048575 non-null datetime64[ns]
End_Time                 1048575 non-null datetime64[ns]
Start_Lat                1048575 non-null float64
Start_Lng                1048575 non-null float64
Distance(mi)             1048575 non-null float64
Description              1048575 non-null object
Street                   1048575 non-null object
Side                     1048575 non-null object
City                     1048552 non-null object
County                   1048575 non-null object
State                    1048575 non-null object
Zipcode                  1048430 non-null object
Country                  1048575 non-null object
Timezone                 1047966 non-null object
Airport_Code             1047456 non-

In [4]:
#One hot encoding columns
bool_cols = list(a_data.select_dtypes(include = bool).columns)
times_of_day = {'Day':1, 'Night':0}
day_cols = ['Sunrise_Sunset','Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

id_dict = {True:1, False:0}
for col in bool_cols:
    a_data[col] = a_data[col].map(id_dict)

for day in day_cols:
    a_data[day] = a_data[day].map(times_of_day)

In [5]:
"""
Assigning times of day and seeing what's left over. Setting all the missing times of day columns to be the same, extremely
likely I only keep 1.
"""
missing_times = a_data.loc[a_data['Sunrise_Sunset'].isnull()]

for i, row in missing_times.iterrows():
    if row['Start_Time'].hour >= 8 and row['Start_Time'].hour <= 16:
        a_data.loc[i, 'Sunrise_Sunset':'Astronomical_Twilight'] = 1
    elif row['Start_Time'].hour <= 4 or row['Start_Time'].hour >= 22:
        a_data.loc[i, 'Sunrise_Sunset':'Astronomical_Twilight'] = 0
        
a_data.loc[a_data['Sunrise_Sunset'].isnull()][['Start_Time', 'City', 'State', 'Start_Lat', 'Start_Lng']]
#16 rows. Can fill in manually

,Start_Time,City,State,Start_Lat,Start_Lng
85973,2016-08-02 18:18:00,NaN,CA,34.451862,-117.660103
119777,2016-12-23 19:30:00,NaN,FL,27.388653,-82.441948
122934,2017-01-17 17:34:00,NaN,FL,27.387951,-82.440239
123707,2017-01-24 07:30:00,NaN,FL,27.388653,-82.441948
126323,2016-11-07 06:53:00,NaN,FL,27.388653,-82.441948
132635,2016-09-28 17:25:00,NaN,FL,27.387770,-82.439575
145312,2016-07-25 17:58:00,NaN,FL,27.388733,-82.445129
242448,2016-11-17 05:02:00,NaN,MD,38.967533,-77.077103
444987,2017-05-17 19:19:00,NaN,WI,42.496330,-88.992889
498104,2017-07-10 20:27:00,NaN,FL,27.388653,-82.441948


In [6]:
#Filling in the rest. Can create a correct sized matrix from these values and use them for assignment
final_missing_times = [1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1]
missing_indices = a_data.loc[a_data['Sunrise_Sunset'].isnull()].index

for col in day_cols:
    a_data.loc[missing_indices, col] = final_missing_times

a_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 46 columns):
Source                   1048575 non-null object
TMC                      1048575 non-null int64
Severity                 1048575 non-null int64
Start_Time               1048575 non-null datetime64[ns]
End_Time                 1048575 non-null datetime64[ns]
Start_Lat                1048575 non-null float64
Start_Lng                1048575 non-null float64
Distance(mi)             1048575 non-null float64
Description              1048575 non-null object
Street                   1048575 non-null object
Side                     1048575 non-null object
City                     1048552 non-null object
County                   1048575 non-null object
State                    1048575 non-null object
Zipcode                  1048430 non-null object
Country                  1048575 non-null object
Timezone                 1047966 non-null object
Airport_Code             1047456 non-

In [7]:
a_data[a_data['City'].isnull()][['City', 'State', 'Start_Lat', 'Start_Lng']]
#From manual lookup: CA is Llano, FL is Bradenton, MD is Chevy Chase, WI is Beloit

,City,State,Start_Lat,Start_Lng
85973,NaN,CA,34.451862,-117.660103
111085,NaN,CA,34.451862,-117.660103
119777,NaN,FL,27.388653,-82.441948
122934,NaN,FL,27.387951,-82.440239
123707,NaN,FL,27.388653,-82.441948
125053,NaN,FL,27.388653,-82.441948
126323,NaN,FL,27.388653,-82.441948
126456,NaN,FL,27.388653,-82.441948
127218,NaN,FL,27.388653,-82.441948
132635,NaN,FL,27.387770,-82.439575


In [8]:
a_data.loc[(a_data['City'].isnull()) & (a_data['State'] == 'CA'), 'City'] = 'Llano'
a_data.loc[(a_data['City'].isnull()) & (a_data['State'] == 'FL'), 'City'] = 'Bradenton'
a_data.loc[(a_data['City'].isnull()) & (a_data['State'] == 'MD'), 'City'] = 'Chevy Chase'
a_data.loc[(a_data['City'].isnull()) & (a_data['State'] == 'WI'), 'City'] = 'Beloit'

In [9]:
#Filling in missing weather timestamps with accident timestamps. Very likely I won't keep weather timestamp as a feature
a_data.loc[a_data['Weather_Timestamp'].isnull(), 'Weather_Timestamp'] = a_data.loc[a_data['Weather_Timestamp'].isnull(), 'Start_Time']

In [10]:
#Loading zipcodes data
search = SearchEngine(simple_zipcode=True)

In [11]:
#Missing zip codes
missing_zip_lats = a_data[a_data['Zipcode'].isnull()]['Start_Lat']
missing_zip_lons = a_data[a_data['Zipcode'].isnull()]['Start_Lng']

found_zips = [search.by_coordinates(lat,lon)[0].zipcode for lat, lon in zip(missing_zip_lats, missing_zip_lons)]

a_data.loc[a_data['Zipcode'].isnull(), 'Zipcode'] = found_zips

In [12]:
#Missing timezones
tf = TimezoneFinder()
missing_tz_lats = a_data[a_data['Timezone'].isnull()]['Start_Lat']
missing_tz_lons = a_data[a_data['Timezone'].isnull()]['Start_Lng']

found_tz = [tf.timezone_at(lat = lt, lng = ln) for lt, ln in zip(missing_tz_lats, missing_tz_lons)]
a_data.loc[a_data['Timezone'].isnull(), 'Timezone'] = found_tz

In [ ]:
#Other data cleaning I am trying to do will be easier in excel
a_data.to_csv(r'C:\Users\Mikhail\Documents\Data 606\US_Accidents_Dec19_stage1.csv')